In [172]:
import numpy as np
import cv2
import copy
import math
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
from sklearn.feature_extraction import image
from sklearn.tree import DecisionTreeClassifier
from numpy.random import choice

In [141]:
import warnings
warnings.filterwarnings('ignore')

In [142]:
os.chdir('..')

In [143]:
os.chdir('Q3-faces')

In [144]:
size_local=255

In [137]:
# Load an color image 
img = cv2.imread('face3.jpg',1)
cv2.imshow('Main Image ',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Q3 A Skin color thresholding

In [138]:
 img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
upper_limit = np.array([20, 255, 255])
lower_limit = np.array([0, 48, 80])

mask1 = cv2.inRange(img, lower_limit, upper_limit)

mask1 = cv2.GaussianBlur(mask1, (3, 3), 0)
skin = cv2.bitwise_and(img, img, mask=mask1)
img1=cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)

In [139]:
# Load an color image 

cv2.imshow('After Skin thresholding ',img1)
cv2.imwrite('After_skin_threshold1.png',(img1/np.max(img1)*size_local))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [79]:
def patch_calc(train,size):
    r=int(size/2)
    split_h=np.hsplit(train,r)
    splits_main=[]
    for i in range(len(split_h)):
        split_v=np.vsplit(split_h[i],r)
        splits_main+=split_v
    return np.array(splits_main)

In [109]:
np.array(train_data).shape

(400, 200, 200)

# Q3 B Adaboosting Face detector

In [188]:
def make_train_data(train_data,feature_value):
    train_data1=[]
    if feature_value==0:
        for i in range(len(train_data)):
            temp=[]
            for j in range(20,len(train_data[i]),20):
                for k in range(40,len(train_data[i][j]),40):
                    try:
                        temp.append((train_data[i][j][k]-train_data[i][j-20][k])-(train_data[i][j][k-40]-train_data[i][j-20][k-40]))
                    except:
                        pass
            train_data1.append(temp)
    elif feature_value==1:
        for i in range(len(train_data)):
            temp=[]
            for j in range(20,len(train_data[i]),20):
                for k in range(40,len(train_data[i][j]),40):
                    try:
                        temp.append((train_data[i][j][k-40]-train_data[i][j-20][k-40])-(train_data[i][j][k]-train_data[i][j-20][k]))
                    except:
                        pass
            train_data1.append(temp)
    elif feature_value==2:
        for i in range(len(train_data)):
            temp=[]
            for j in range(10,len(train_data[i]),10):
                for k in range(10,len(train_data[i][j]),20):
                    try:
                        temp.append(((train_data[i][j][k+10]-train_data[i][j-10][k+10])-(train_data[i][j][k]-train_data[i][j-10][k]))-(((train_data[i][j][k]-train_data[i][j-10][k])-(train_data[i][j][k-10]-train_data[i][j-10][k-10]))+((train_data[i][j][k+20]-train_data[i][j-10][k+20])-(train_data[i][j][k+10]-train_data[i][j-10][k+10]))))
                    except:
                        pass
            train_data1.append(temp)
    elif feature_value==3:
        for i in range(len(train_data)):
            temp=[]
            for j in range(10,len(train_data[i]),10):
                for k in range(10,len(train_data[i][j]),20):
                    try:
                        temp.append((((train_data[i][j][k]-train_data[i][j-10][k])-(train_data[i][j][k-10]-train_data[i][j-10][k-10]))+((train_data[i][j][k+20]-train_data[i][j-10][k+20])-(train_data[i][j][k+10]-train_data[i][j-10][k+10])))-((train_data[i][j][k+10]-train_data[i][j-10][k+10])-(train_data[i][j][k]-train_data[i][j-10][k])))
                    except:
                        pass
            train_data1.append(temp)
    
    return train_data1
        

In [189]:
def calculate_feature(train_data,feature,threshold):
    train_data1=[]
    temp=[0]*4
    for i in range(len(train_data)):
       
        for j in range(20,len(train_data[i]),20):
            for k in range(40,len(train_data[i][j]),40):
                try:
                    if (train_data[i][j][k]-train_data[i][j-20][k])-(train_data[i][j][k-40]-train_data[i][j-20][k-40])>threshold:
                        temp[0]+=1
                except:
                    pass
 
   
    for i in range(len(train_data)):
   
        for j in range(20,len(train_data[i]),20):
            for k in range(40,len(train_data[i][j]),40):
                try:
                    if (train_data[i][j][k-40]-train_data[i][j-20][k-40])-(train_data[i][j][k]-train_data[i][j-20][k])>threshold:
                        temp[1]+=1
                except:
                    pass
  
    for i in range(len(train_data)):

        for j in range(10,len(train_data[i]),10):
            for k in range(10,len(train_data[i][j]),20):
                try:
                    if (((train_data[i][j][k+10]-train_data[i][j-10][k+10])-(train_data[i][j][k]-train_data[i][j-10][k]))-(((train_data[i][j][k]-train_data[i][j-10][k])-(train_data[i][j][k-10]-train_data[i][j-10][k-10]))+((train_data[i][j][k+20]-train_data[i][j-10][k+20])-(train_data[i][j][k+10]-train_data[i][j-10][k+10]))))>threshold:
                        temp[2]+=1
                except:
                    pass
   
    for i in range(len(train_data)):

        for j in range(10,len(train_data[i]),10):
            for k in range(10,len(train_data[i][j]),20):
                try:
                    if ((((train_data[i][j][k]-train_data[i][j-10][k])-(train_data[i][j][k-10]-train_data[i][j-10][k-10]))+((train_data[i][j][k+20]-train_data[i][j-10][k+20])-(train_data[i][j][k+10]-train_data[i][j-10][k+10])))-((train_data[i][j][k+10]-train_data[i][j-10][k+10])-(train_data[i][j][k]-train_data[i][j-10][k])))>threshold:
                        temp[3]+=1
                except:
                    pass
                
    return np.argmax(np.array(temp))
       

In [208]:
def adaboost(n,train_data1,train_label1,test_data,test_label,weights,d):
    alpha_k=[]
    Ck=[]
    nat=[i for i in range(len(train_data1))]
    main_data=copy.deepcopy(train_data1)
    main_label=copy.deepcopy(train_label1)
#     patches=[]
#     for i in range(len(train_data1)):
#         pat=patch_calc(train_data1[i],10)
#         patches.append(pat)
    main_features_set=[]
    for i in range(n):
        sample = choice(nat, d,p=weights,replace=False)
        train_data=[]
        train_label=[]
        for j in range(len(sample)):
            train_data.append(train_data1[sample[j]])
            train_label.append(train_label1[sample[j]])
        features=np.array([0]*4)
        for j in range(4):
            
            
            features[calculate_feature(train_data,j,700)]+=1
            
        feature_val=np.argmax(features)
        main_features_set.append(feature_val)
        train_data1=make_train_data(main_data,feature_val)
        train_data2=make_train_data(train_data,feature_val)
        print(len(train_data1))
        clf=DecisionTreeClassifier(max_depth=2,max_leaf_nodes=5)
        clf.fit(np.array(train_data1),np.array(main_label))
        predict1=clf.predict(np.array(train_data2))
        h=labelling(predict1.tolist(),train_label)
        train_err=clf.score(np.array(train_data2),np.array(train_label))
        train_err=1-train_err
        alpha=0.5*np.log((1-train_err)/float(train_err))+np.log(25)
        alpha_k.append(alpha)
        Ck.append(clf)
#         print("Hello1 : ",i)
        for j in range(len(weights)):
            
            if h[j]==1:
                
                weights[j]=weights[j]*math.exp((-1)*alpha)
            else:
                weights[j]=weights[j]*math.exp(alpha)
        w=copy.deepcopy(weights)
        total=np.sum(w)
        for j in range(len(weights)):
            weights[j]=weights[j]/float(total)
#         print("Hello2 : ",i)
 #For test set
    test_predict=[]
    for i in range(len(test_data)):
        disc_func=[[] for i in class_label]
        for j in range(k_max):
            index=Ck[j].predict(np.array(test_data[i]).reshape(1,-1)).tolist()[0]
            if disc_func[index]==[]:
                disc_func[index].append(alpha_k[j])
            else:
                disc_func[index][0]+=alpha_k[j]
        test_predict.append(disc_func.index(max(disc_func)))

    test_acc=accuracy(test_predict,test_label)   
# For train set
    train_predict=[]
    for i in range(len(train_data1)):
        disc_func1=[[] for i in class_label]
        for j in range(k_max):
            index=Ck[j].predict(np.array(train_data1[i]).reshape(1,-1)).tolist()[0]
            if disc_func1[index]==[]:
                disc_func1[index].append(alpha_k[j])
            else:
                disc_func1[index][0]+=alpha_k[j]
        train_predict.append(disc_func1.index(max(disc_func1)))

    train_acc=accuracy(train_predict,train_label1)  
    
    return Ck,alpha_k,train_predict,test_predict,train_acc,test_acc,main_features_set
            

In [184]:
pwd

'C:\\Users\\Dell-pc\\Desktop\\CV3\\Q3-faces\\Data_Set'

In [149]:
os.chdir('Data_Set')

In [150]:
os.listdir()

['Face', 'Non_face']

In [151]:
train_data=[]
train_label=[]
os.chdir('Face')
list_dir=os.listdir()
for i in list_dir:
    img=cv2.imread(i,0)
    img=cv2.resize(img,(199,199))
    img1=cv2.integral(img)
    train_data.append(img1)
    train_label.append(1)

os.chdir('..')
os.chdir('Non_face')
list_dir=os.listdir()
for i in list_dir:
    img=cv2.imread(i,0)
    img=cv2.resize(img,(199,199))
    img1=cv2.integral(img)
    train_data.append(img1)
    train_label.append(0)
os.chdir('..')

In [155]:
main_data=[]
for i in range(len(train_data)):
    temp=[]
    temp.append(train_data[i])
    temp.append(train_label[i])
    main_data.append(temp)
    
    

In [156]:
data=random.sample(main_data,len(main_data))

In [157]:
X_train=data[:400]
X_test=data[400:]

In [158]:
train_data=[]
train_label=[]
test_data=[]
test_label=[]
for i in range(len(X_train)):
    train_data.append(X_train[i][0])
    train_label.append(X_train[i][1])
for i in range(len(X_test)):
    test_data.append(X_test[i][0])
    test_label.append(X_test[i][1])

In [209]:
weights=[1/float(len(train_data)) for i in train_data]
k_max=10
d=50

In [199]:
def accuracy(predict,true):
    count=0
    for i in range(len(predict)):
        if predict[i]==(true[i]):
            count+=1
    return count/float(len(predict))

In [198]:
def labelling(predict,true):
    h=[]
    for i in range(len(predict)):
        if predict[i]==true[i]:
            h.append(1)
        else:
            h.append(0)
    return h

In [210]:
Ck,alpha_k,train_result,test_result,train_acc,test_acc,feature_set=adaboost(k_max,train_data,train_label,test_data,test_label,weights,d)

400


IndexError: list index out of range

# Seeded Segmentation

In [420]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [421]:
gray=cv2.resize(gray,(200,200))

In [422]:
# Point in Face region for face1 it is (100,100) and for face2, it will be(195,100) and for face3 , it is (195,145) and for face 4 it is (100,100)
point=(195,145)

In [423]:
def neighbours(point,r,c):
    temp=[]
    try:
        temp.append(((point[0]-1),(point[1]-1)))
    except:
        pass

    try:
        temp.append(((point[0]-1),(point[1])))
    except:
        pass
    try:
        temp.append(((point[0]-1),(point[1]+1)))
    except:
        pass
    try:
        temp.append(((point[0]),(point[1]-1)))
    except:
        pass
    try:
        temp.append(((point[0]),(point[1]+1)))
    except:
        pass
    try:
        temp.append(((point[0]+1),(point[1]-1)))
    except:
        pass
    try:
        temp.append(((point[0]+1),(point[1])))
    except:
        pass
    try:
        temp.append(((point[0]+1),(point[1]+1)))
    except:
        pass
    temp1=copy.deepcopy(temp)
    for i in range(len(temp)):
        if temp[i][0]<0 or temp[i][1]<0 or temp[i][0]>=r or temp[i][1]>=c:
            del temp1[temp1.index(temp[i])]
    return temp1

In [424]:
point

(195, 145)

In [425]:
main=copy.deepcopy(gray)

In [426]:
main[point]=255

In [427]:
stack=[]
av=gray[point]
stack.append(point)
counter=0
global_stack=copy.deepcopy(stack)
while stack!=[]:
    counter+=1
    if counter%500==0:
        print("Counter : ",counter)
    neigh=neighbours(point,len(gray),len(gray[0]))
#     print("FDTH",neigh)
    sum=0
    for i in range(len(stack)):
        sum+=gray[stack[i]]
    
    av=sum/float(len(stack))
#     print(av)
    for i in neigh:
        if i not in global_stack:
            if abs(gray[i]-av)<15:
    #             print("dskfbsdjf")
                stack.append((i))
                global_stack.append((i))
                main[i]=0
#     print(stack)
    point=stack[0]
    del stack[0]
    
        
    
    

Counter :  500
Counter :  1000
Counter :  1500
Counter :  2000
Counter :  2500
Counter :  3000
Counter :  3500
Counter :  4000
Counter :  4500
Counter :  5000
Counter :  5500
Counter :  6000
Counter :  6500
Counter :  7000
Counter :  7500
Counter :  8000
Counter :  8500
Counter :  9000
Counter :  9500
Counter :  10000
Counter :  10500
Counter :  11000
Counter :  11500
Counter :  12000


In [428]:
# Load an color image 

cv2.imshow('After Seeded ',main)
cv2.imwrite('After_seeded_seg3.png',(main/np.max(main)*size_local))
cv2.waitKey(0)
cv2.destroyAllWindows()